<a href="https://colab.research.google.com/github/taoyafan/cnn-dailymail/blob/master/make_data_file_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

教程参考：

[想免费用谷歌资源训练神经网络？Colab 详细使用教程](https://jinkey.ai/post/tech/xiang-mian-fei-yong-gu-ge-zi-yuan-xun-lian-shen-jing-wang-luo-colab-xiang-xi-shi-yong-jiao-cheng#toc_2)

[Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)

# 重启虚拟机

虚拟机上所有的东西都会消失

In [0]:
# !kill -9 -1

# 授权登录、挂载云盘

In [62]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

drive.mount("/content/drive")
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 将数据从云盘读入虚拟机根目录

## 文件名和路径设置

是否自动生成测试文件？如果是，则从训练集中抽出一部分作为测试集，否则手动设定测试集

In [0]:
AUTO_TEST = True

if AUTO_TEST:
    TEST_NUM = 20

In [64]:
import time
import os
import shutil

stories_dir = ""
split_stories_dir = "split_stories"
tokenized_stories_dir = "tokenized_stories"
finished_files_dir = "finished_files"
drive_finished_files_dir = "drive/My Drive/code/get_to_the_point/finished_files"
smrz_dir = "drive/My Drive/code/get_to_the_point/pointer-generator-master"
chunks_dir = os.path.join(finished_files_dir, "chunked")
drive_chunks_dir = os.path.join(drive_finished_files_dir, "chunked")


# file_names = ["bytecup.corpus.train.{}.txt".format(i) for i in range(8,9)]
train_file_names = ["bytecup.corpus.train.0.txt"]
val_file_names = ["bytecup.corpus.validation_set.txt"]
if not AUTO_TEST:
    test_file_names = ["for_test.txt"]

# if not os.path.exists(stories_dir): raise Exception("stories_dir %s doesn't exist." % (stories_dir))
if not os.path.exists(split_stories_dir): os.makedirs(split_stories_dir)
if not os.path.exists(tokenized_stories_dir): os.makedirs(tokenized_stories_dir)
if not os.path.exists(finished_files_dir): os.makedirs(finished_files_dir)
if not os.path.exists(drive_finished_files_dir): os.makedirs(drive_finished_files_dir)
if not os.path.exists(chunks_dir): os.makedirs(chunks_dir)
# if not os.path.exists(drive_chunks_dir): os.makedirs(drive_chunks_dir)
if not os.path.exists(smrz_dir): raise Exception("smrz_dir %s doesn't exist." % (smrz_dir))
    
print("train_file_names: %s"% train_file_names)
print("val_file_names: %s"% val_file_names)
if not AUTO_TEST:
    print("test_file_names: %s"% test_file_names)
else:
    print("test_file_names: %s"% train_file_names)

train_file_names: ['bytecup.corpus.train.0.txt']
val_file_names: ['bytecup.corpus.validation_set.txt']
test_file_names: ['bytecup.corpus.train.0.txt']


## 复制数据原始文件（file_names）

In [65]:
drive_dir = "drive/My Drive/code"

def copy_file(file_names, drive_dir):
    for file_name in file_names:
        src_file = os.path.join(drive_dir, file_name)
        print("Copy file from \"%s\" to \"%s\""% (src_file, file_name))
        shutil.copyfile(src_file, file_name) 

copy_file(train_file_names, drive_dir)
copy_file(val_file_names, drive_dir)
if not AUTO_TEST:
    copy_file(test_file_names, drive_dir)


Copy file from "drive/My Drive/code/bytecup.corpus.train.0.txt" to "bytecup.corpus.train.0.txt"
Copy file from "drive/My Drive/code/bytecup.corpus.validation_set.txt" to "bytecup.corpus.validation_set.txt"


## 原始方法，不需要授权登录

显示云盘根目录的文件

In [0]:
# file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))


打开 code 文件

In [0]:
# file_list = drive.ListFile({'q': "'18ijhbBiNtaqpgiNwzxAf6a3W13Pc8qSa' in parents and trashed=false"}).GetList()
# for file1 in file_list:
#       print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))
  

读取云盘中的文件内容

In [0]:
def get_file_content(file_name, show = False):
    id = [file['id'] for file in file_list if file['title'] == file_name][0]
  
    if id == []:
        print("There are not a file named %s"% file_name)
        return []
  
    print("Matched file named %s, and the id is %s"% (file_name, id))
    file = drive.CreateFile({'id': id})
    txt = file.GetContentString()
    if show:
        print("This is the content of file %s :\n\n"% file_name)
        print(txt)
    
    print("Read file complete, length of file is %d" % len(txt))
    return txt
  
# txt = get_file_content(file_name, show = False)

内容写入虚拟机

In [0]:
# import os
# with open(file_name, 'w') as f:
#     f.write(txt)
# !ls

读取根目录文件，测试写入成功

In [0]:
def show_file(str):
    with open(str, 'r') as load_f:
        print("This is the content of file %s :\n\n"% str)
        lines = load_f.readlines()
        for i, line in enumerate(lines):
            print(line)
        print("\n\n")
    
# show_file(file_name)

删除文件，例如

remove_files(os.listdir(dir))

可以删除路径dir下所有文件

In [0]:
def remove_files(file_names):
    for file_name in file_names:
        os.remove(file_name)

# 开始对数据进行预处理

## 将每个数据集文件拆分个多个小文件
每个小文件包含一个故事的内容和标题，以 ***数据类型_ ID***  命名，如 ***train_1*** 存放在 split_stories_dir 下

In [0]:
import json
import os
import re
import random

def split_lines(lines, story_type):
    '''
        输入参数 lines 是多行的数据文件，每行是一个样本，样本类型为 story_type
        
        将输入的 lines 按行拆分，分别写入文件，文件名为 story_type_id
    '''
    print('Writting %s file in %s...'% (story_type, split_stories_dir))
    
    # 读取数据集的每一个样本
    for i, line in enumerate(lines):
        txt_dic = json.loads(line)
        split_file_dir = os.path.join(split_stories_dir, "%s_%s"%(story_type, txt_dic['id']))

        # 将每个样本单独写入一个文件中
        with open(split_file_dir, "w") as f:
            txt_dic['content'] = re.sub(r"(?<=[a-zA-Z\"\'\[\]\(\)\-\$]{2})[.](?=\S)|[.](?=[a-zA-Z\"\'\[\]\(\)\-\$]{2})",
                                        ". ", txt_dic['content']) 
            f.write(txt_dic['content'])
            f.write(' . ')
            if('title' in txt_dic.keys()):
                f.write("\n\n@title.\n")
                f.write(txt_dic['title'])


def split_story_files(file_names, file_type):
    for name in file_names:
        file_dir = os.path.join(stories_dir, name)
        heads = []
        desc = []

        print('Opening file: %s...' % name)
        time_start = time.time()

        # 打开每一个数据集
        with open(file_dir, 'r') as load_f:
            lines = load_f.readlines()
        
        if AUTO_TEST and file_type == 'train':
#             random.shuffle(lines)
            split_lines(lines[: TEST_NUM], 'test')
            split_lines(lines[TEST_NUM : -1], 'train')
        else:
            split_lines(lines, file_type)

        print("completed")
        time_end = time.time()
        print('Consume time:', time_end - time_start, '\n')

In [71]:
# shutil.rmtree(split_stories_dir)
# os.makedirs(split_stories_dir)

split_story_files(train_file_names, "train")
split_story_files(val_file_names, "val")

if not AUTO_TEST:
    split_story_files(test_file_names, "test")
    


Opening file: bytecup.corpus.train.0.txt...
Writting test file in split_stories...
Writting train file in split_stories...
completed
Consume time: 59.97026228904724 

Opening file: bytecup.corpus.validation_set.txt...
Writting val file in split_stories...
completed
Consume time: 0.30092930793762207 



## 对每篇文章进行分词（分句）

### 安装 java 添加环境变量

In [72]:
time_start = time.time()

!apt-get install openjdk-8-jdk
!java -version
os.environ['CLASSPATH']="/content/drive/My Drive/pack/stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar"

time_end = time.time()
print('Consume time:', time_end - time_start)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk is already the newest version (8u181-b13-1ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
openjdk version "1.8.0_181"
OpenJDK Runtime Environment (build 1.8.0_181-8u181-b13-1ubuntu0.18.04.1-b13)
OpenJDK 64-Bit Server VM (build 25.181-b13, mixed mode)
Consume time: 4.4622344970703125


### 选择是否分句
若为 DIVIDE_SENTENCE = True 则分句加分词，否则只分词

In [0]:
DIVIDE_SENTENCE = False

### 只分词不分句
先生成映射文件 mapping，再使用 PTBTokenizer 对文件进行直接操作

In [74]:

import subprocess

if not DIVIDE_SENTENCE:

    time_start = time.time()
    
    stories = os.listdir(split_stories_dir)
    #生成映射文件 mapping.txt
    print("Making list of files to tokenize...")
    with open("mapping.txt", "w") as f:
        for s in stories:
            f.write("%s \t %s\n" % (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))
    print("completed\n")
    
    print("Token file frome \"%s\" to \"%s\""% (split_stories_dir, tokenized_stories_dir))
    # Doesn't work
    # command = ['java', 'edu.stanford.nlp.process.DocumentPreprocessor', '-ioFileList', '-preserveLines', 'mapping.txt']
    
    command = ['java', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
    subprocess.call(command)
    
    # Doesn't work
    # os.system("java edu.stanford.nlp.process.DocumentPreprocessor -ioFileList -preserveLines mapping.txt")
    
    time_end = time.time()
    print('Consume time:', time_end - time_start)
    os.remove("mapping.txt")

Making list of files to tokenize...
completed

Token file frome "split_stories" to "tokenized_stories"
Consume time: 61.52289152145386


### 分句加分词
无法利用映射文件进行分词，有BUG，故只能一个一个文件分词

In [0]:
import os
if DIVIDE_SENTENCE:
    
    time_start = time.time()
    
    stories = os.listdir(split_stories_dir)
    for s in stories:
        os.system("java edu.stanford.nlp.process.DocumentPreprocessor -preserveLines < %s > %s" %
                  (os.path.join(split_stories_dir, s), os.path.join(tokenized_stories_dir, s)))

    time_end = time.time()
    print('Consume time:', time_end - time_start)

## 格式化文章并存储

### （函数）根据文件地址读文件，返回行列表

In [0]:
def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
        return lines

### （函数）根据文件内容获得文章和摘要

In [0]:
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'

def get_art_abs(story_file):
    lines = read_text_file(story_file)

    # Lowercase everything
    lines = [line.lower() for line in lines]

    # Separate out article and abstract sentences
    article_lines = []
    titles = []
    next_is_title = False
    for idx,line in enumerate(lines):
        if line == "":
            continue # empty line
        elif line.startswith("@title"):
            next_is_title = True
        elif next_is_title:
            titles.append(line)
        else:
            article_lines.append(line)

    # Make article into a single string
    article = ' '.join(article_lines)

    # Make abstract into a signle string, putting <s> and </s> tags around the sentences
    title = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in titles])

    return article, title

### 写入 bin 文件

In [0]:
from tensorflow.core.example import example_pb2
import struct
import glob

def write_to_bin(src_dir, out_dir, file_type):

    time_start = time.time()

    out_file = os.path.join(out_dir, '%s.bin'% file_type)
    print("Writing file %s.bin to %s..."% (file_type, out_dir) )
    with open(out_file, 'wb') as writer:
        stories = glob.glob(os.path.join(tokenized_stories_dir, '%s_*'% file_type))
#         stories = os.listdir(src_dir)
#         print(stories)
        for idx, s in enumerate(stories):
#             story_file = os.path.join(tokenized_stories_dir, s)
            article, title = get_art_abs(s)
#             print("article: \n%s\n\n"%article)
#             print("title: \n%s\n" % title)

            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
            tf_example.features.feature['abstract'].bytes_list.value.extend([title.encode()])
            tf_example_str = tf_example.SerializeToString()

            str_len = len(tf_example_str)
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str))

    time_end = time.time()
    print('Consume time:', time_end - time_start)


In [79]:
write_to_bin(tokenized_stories_dir, finished_files_dir, 'train')
write_to_bin(tokenized_stories_dir, finished_files_dir, 'test')
write_to_bin(tokenized_stories_dir, finished_files_dir, 'val')

Writing file train.bin to finished_files...
Consume time: 23.602994918823242
Writing file test.bin to finished_files...
Consume time: 0.22115588188171387
Writing file val.bin to finished_files...
Consume time: 0.37483882904052734


In [0]:
# stories = os.listdir(tokenized_stories_dir)
# remove_files(os.listdir(tokenized_stories_dir))
# stories = os.listdir(tokenized_stories_dir)


### 将文件分块，每个块的文章个数为 CHUNK_SIZE

In [80]:
CHUNK_SIZE = 5000 # num examples per chunk, for the chunked data 1000

def chunk_file(set_name):
        
    in_file = os.path.join(finished_files_dir, '%s.bin' % set_name)
    
    if not os.path.exists(in_file):
        print("File  %s not exist"% in_file)
        return
    
    with open(in_file, 'rb') as reader:
        chunk = 0
        finished = False
        while not finished:
            chunk_fname = os.path.join(chunks_dir, '%s_%03d.bin' % (set_name, chunk)) # new chunk
            with open(chunk_fname, 'wb') as writer:
                for _ in range(CHUNK_SIZE):
                    len_bytes = reader.read(8)
                    if not len_bytes:
                        finished = True
                        break
                    str_len = struct.unpack('q', len_bytes)[0]
                    example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                    writer.write(struct.pack('q', str_len))
                    writer.write(struct.pack('%ds' % str_len, example_str))
                chunk += 1
            
def chunk_all():
    # Chunk the data
    for set_name in ['train', 'val', 'test']:
#     for set_name in ['test']:
        print("Splitting %s data into chunks..." % set_name)
        chunk_file(set_name)
    print("Saved chunked data in %s" % chunks_dir)
    
# 分块
chunk_all()

Splitting train data into chunks...
Splitting val data into chunks...
Splitting test data into chunks...
Saved chunked data in finished_files/chunked


## 替换虚拟机的文件到云盘中

替换 chunk 文件夹中的文件

In [81]:
def copy_files(src_dir, des_dir):
    names = os.listdir(src_dir)
    num = 0
    for name in names:
        src_file = os.path.join(src_dir, name)
        des_file = os.path.join(des_dir, name)
        shutil.copyfile(src_file, des_file) 
        num += 1
    
    print("Copied %d files from \n%s \nto \n%s"% (num, src_dir, des_dir))
    
if drive_chunks_dir != chunks_dir :
    if os.path.exists(drive_chunks_dir): 
        shutil.rmtree(drive_chunks_dir)
        
    os.makedirs(drive_chunks_dir)
    copy_files(chunks_dir, drive_chunks_dir)

Copied 32 files from 
finished_files/chunked 
to 
drive/My Drive/code/get_to_the_point/finished_files/chunked


# 运行预测程序

In [82]:
!pip install pyrouge

    100% |████████████████████████████████| 61kB 2.6MB/s 
  Running setup.py bdist_wheel for pyrouge ... - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/75/d3/0c/e5b04e15b6b87c42e980de3931d2686e14d36e045058983599
Successfully built pyrouge


## （函数）执行 shell 命令

输入和在 shell 相同的命令，如路径中的空格前需要加"\"

In [0]:
import subprocess

def shell_command(str):
    
    print(str)
    try:
        out_bytes = subprocess.check_output(str, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code

    out_text = out_bytes.decode('utf-8')
    print(out_text)

## 设置模式、路径、名称等参数

In [84]:
# mode = "decode"
mode = "train"
data_path = "/content/finished_files/chunked/train_*"
vocab_path = "/content/drive/My Drive/code/get_to_the_point/finished_files/vocab"
log_root = "/content/log"
drive_log_root = "/content/drive/My Drive/code/get_to_the_point/log"
exp_name = "data0_train"
exp_dir = os.path.join(log_root, exp_name)
drive_exp_dir = os.path.join(drive_log_root, exp_name)
single_pass = "True"
batch_size = "256"
summarization_file = os.path.join(smrz_dir, "run_summarization.py")

drive_train_dir = "/content/drive/My Drive/code/get_to_the_point/log/train"

print(mode)
print(data_path)
print(vocab_path)
print(log_root)
print(exp_name)
print(summarization_file)

if not os.path.exists(log_root): os.makedirs(log_root)


train
/content/finished_files/chunked/train_*
/content/drive/My Drive/code/get_to_the_point/finished_files/vocab
/content/log
data0_train
drive/My Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py


## 复制 train 文件到虚拟机

In [0]:
# shutil.rmtree(exp_dir)

In [0]:
if not os.path.exists(exp_dir):
    if os.path.exists(drive_exp_dir):
        shutil.copytree(drive_exp_dir, exp_dir)
    else:
        os.makedirs(exp_dir)
        shutil.copytree(os.path.join(drive_log_root, "train"), os.path.join(exp_dir, "train"))

## 产生可执行的命令，复制在下一个 cell 执行即可

In [86]:
def get_command():
    if(mode != "decode"):
        sp = "False"
    else:
        sp = single_pass
        
    print("!python3 %s --mode=%s --data_path=%s --vocab_path=%s --log_root=%s --exp_name=%s --single_pass=%s --batch_size=%s" 
                  % (summarization_file.replace(' ', '\ '), mode, data_path, vocab_path.replace(' ', '\ '), log_root, exp_name, sp, batch_size))

get_command()

!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=train --data_path=/content/finished_files/chunked/train_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=False --batch_size=256


In [0]:
# shell_command("python3 %s --mode=%s --data_path=%s --vocab_path=%s --log_root=%s --exp_name=%s --single_pass=%s" 
#               % (summarization_file.replace(' ', '\ '), mode, data_path, vocab_path.replace(' ', '\ '), log_root, exp_name, single_pass))
!python3 drive/My\ Drive/code/get_to_the_point/pointer-generator-master/run_summarization.py --mode=train --data_path=/content/finished_files/chunked/train_* --vocab_path=/content/drive/My\ Drive/code/get_to_the_point/finished_files/vocab --log_root=/content/log --exp_name=data0_train --single_pass=False --batch_size=256

INFO:tensorflow:Starting seq2seq_attention in train mode...








max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: cardholders
HParams(mode='train', hidden_dim=256, emb_dim=128, batch_size=256, max_enc_steps=400, max_dec_steps=15, lr=0.15, adagrad_init_acc=0.1, rand_unif_init_mag=0.02, trunc_norm_init_std=0.0001, max_grad_norm=2.0, pointer_gen=True, coverage=False, cov_loss_wt=1.0)
creating model...
INFO:tensorflow:Building graph...
Writing word embedding metadata file to /content/log/data0_train/train/vocab_metadata.tsv...
INFO:tensorflow:Adding attention_decoder timestep 0 of 15
INFO:tensorflow:Adding attention_decoder timestep 1 of 15
INFO:tensorflow:Adding attention_decoder timestep 2 of 15
INFO:tensorflow:Adding attention_decoder timestep 3 of 15
INFO:tensorflow:Adding attention_decoder timestep 4 of 15
INFO:tensorflow:Adding attention_decoder timestep 5 of 15
INFO:tensorf

## 解码测试

In [0]:
mode = "decode"
data_path = "/content/finished_files/chunked/test_*"
single_pass = "True"

get_command()

## 复制 log 文件到云盘

In [70]:
if os.path.exists(drive_exp_dir): 
    shutil.rmtree(drive_exp_dir)

shutil.copytree(exp_dir, drive_exp_dir)

'/content/drive/My Drive/code/get_to_the_point/log/data0_train'